In [32]:
import subprocess
import json
from dotenv import load_dotenv
import os

from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3

from eth_account import Account

In [120]:
load_dotenv()
from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [34]:
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

orbit solution donate target dove sock paddle zone double suffer robust call


In [71]:
def derive_wallets(phrase, coin, numderive):
    command = f'php derive -g --mnemonic="{phrase}" --numderive={numderive} --coin={coin} --format=jsonpretty'
    p = subprocess.Popen(command,stdout=subprocess.PIPE,shell=True)
    (output, err) = p.communicate()
    return json.loads(output)

In [72]:
coins = {'eth':derive_wallets(phrase=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnem=mnemonic,coin=BTCTEST,numderive=3)}
print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "mu2db6FcX5MA339FSUzpgVkof8YWCu4k8t",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cRqYF8ZzQd836Dqzry7uC5rzxMyJ7HdqtQfbJhRE7iiUyjZWRSgp",
            "pubkey": "027681403d7aa5226d000575c7c1b4641a2300432c96b33574a96ef371cde7ee9a",
            "pubkeyhash": "94367fb498fa83c3907c7568e66e3186da9fb937",
            "xprv": "tprv8kJPa6TmhymeJ6zBSHPXdJ6toNRZRFaYs52dwPCg4AAKdeFENetcEFpS3TVPri1LyGDRZ2mC7upjeM1eLdZhMPG5oKcidsSvVuR8UarSshZ",
            "xpub": "tpubDGzRiWW1rMTKBa1yKw482hm1NPwVaamTSNdRDuEyURxiU8W113iCQkSJDaNZGG1SUdv2nNHBUKhE5WGjEYkCLoZKCNRP4XvHMrmw9vPBoro"
        },
        {
            "address": "n1vF5RSzaW2jSDJw3uZb4WHWeMGxnTFzgc",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cVoTBSRNoyQ2BfhFs6VT15vnF8q4yWhyVyZWD26bVDKmfJwEfrbD",
            "pubkey": "025a84c0e021bdf16699bb52aa146ea1c6493d535d492625679c6d83ad0143fd06",
            "pubkeyh

In [126]:
eth_key = coins['eth'][1]['privkey']
btc_key = coins['btc-test'][0]['privkey']

In [127]:
def priv_key_to_account(coin, priv_key):
    global account
    if coin == 'eth':
        account=Account.privateKeyToAccount(priv_key)
    else:
        account=PrivateKeyTestnet(priv_key)
    return account

In [131]:
priv_key_to_account(ETH,eth_key)

In [139]:
priv_key_to_account(BTCTEST,btc_key)

<PrivateKeyTestnet: mu2db6FcX5MA339FSUzpgVkof8YWCu4k8t>

In [140]:
def create_tx(coin,account,recipient,amount):
    global tx_data
    if coin =='eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data= {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return tx_data
    else:
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        return tx_data

In [141]:
create_tx(BTCTEST,account,'mu2db6FcX5MA339FSUzpgVkof8YWCu4k8t',0.0001)

'{"unspents":[{"amount":3602698,"confirmations":3,"script":"76a91494367fb498fa83c3907c7568e66e3186da9fb93788ac","txid":"3b2a4482e7c093f73c8c3c0c9b8d75745e9af762a38f2d643dd08533ef6f52c6","txindex":1,"type":"p2pkh","vsize":148,"segwit":false},{"amount":10000,"confirmations":3,"script":"76a91494367fb498fa83c3907c7568e66e3186da9fb93788ac","txid":"3b2a4482e7c093f73c8c3c0c9b8d75745e9af762a38f2d643dd08533ef6f52c6","txindex":0,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["mu2db6FcX5MA339FSUzpgVkof8YWCu4k8t",10000],["mu2db6FcX5MA339FSUzpgVkof8YWCu4k8t",3595218]]}'

Was able to get everything to run and transact, except kept receiving an error message with the send ETH coin before...  not sure why but SO close!

In [1]:
#create_tx(ETH,account,'0x52f820e661cbB14A33CC4dB48422E012F7675fF1',0.0001)

In [106]:
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_data = create_tx(coin,account,recipient,amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed

In [107]:
send_tx(BTCTEST,account,'mu2db6FcX5MA339FSUzpgVkof8YWCu4k8t',0.0001)

'01000000027ae7880c6017700d68a4fe0a01f64d6c1559e9237ea5cbc414741673c86b2229010000006a4730440220575d3222f98508e0d6c7985e8db1bed8624d059e2082ada1990d58330db7bc3202204b34d183d36e998ce1d2418165ef4d368d085819588c53630600f1562b5d55910121027681403d7aa5226d000575c7c1b4641a2300432c96b33574a96ef371cde7ee9affffffff7ae7880c6017700d68a4fe0a01f64d6c1559e9237ea5cbc414741673c86b2229000000006a4730440220682753473bd9458a077d8fd5860e90e3be5cd82ddaedde957a5a39fb95dc39f202207d61996658b8bd7ec51d121b73e7cc99d1a1a00ffb0a1af59a323fb38243d5a60121027681403d7aa5226d000575c7c1b4641a2300432c96b33574a96ef371cde7ee9affffffff0210270000000000001976a91494367fb498fa83c3907c7568e66e3186da9fb93788ac0af93600000000001976a91494367fb498fa83c3907c7568e66e3186da9fb93788ac00000000'

In [101]:
account

<PrivateKeyTestnet: mu2db6FcX5MA339FSUzpgVkof8YWCu4k8t>